In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from collections import defaultdict, Counter
from ds_tools.ds_tools import CategoricalTransformer
import matplotlib.pyplot as plt
import pickle
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
test_surface = df_test['tc_mseki']

df_train.drop(['id', 'kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['id', 'kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

## Preprocessing

In [3]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))

combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [4]:
for col in ['mseki_rd_hb', 'road3_fi', 'rosenka_hb', 'kempei2', 'road2_mg', 'kaoku_hb', 'bus_hon']:
    df_train[col].replace(0.0, np.nan, inplace=True)
    df_test[col].replace(0.0, np.nan, inplace=True)

coordinates = pickle.load(open('./data/coordinates.bin', 'rb'))

for df in [df_train, df_test]:
    df['station_name_prefix'] = df['rosen_nm1'].str.slice(stop=2)
    df.loc[pd.isnull(df['chiseki_kb_hb']) | (df['chiseki_kb_hb'] == 0), 'chiseki_kb_hb'] = df.loc[pd.isnull(df['chiseki_kb_hb']) | (df['chiseki_kb_hb'] == 0), 'chiseki_js_hb']
    df['chiseki_ratio'] = df['chiseki_js_hb']/df['chiseki_kb_hb']
    
    for col in ['magutchi']:
        col_mean = df_train[col].mean()
        df_train[col] = df_train[col].fillna(col_mean)
        df_test[col] = df_test[col].fillna(col_mean)
    
    df['lat'] = df['jukyo'].apply(lambda j: coordinates[j]['results'][0]['geometry']['location']['lat'] if coordinates[j]['results'] else np.nan)
    df['lng'] = df['jukyo'].apply(lambda j: coordinates[j]['results'][0]['geometry']['location']['lng'] if coordinates[j]['results'] else np.nan)
    
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

for df in [df_train, df_test]:
    df['jukyo'] = df['jukyo'].str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)

In [5]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
price_stats = []
for train_idx, valid_idx in splitter.split(df_train):
    price_stats_by_city = defaultdict(dict)
    for city, group in df_train.iloc[train_idx].groupby('city'):
        price_list = group['keiyaku_pr']/group['tc_mseki']
        price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
        price_stats_by_city[city]['price_by_city_median'] = price_list.median()
        price_stats_by_city[city]['price_by_city_min'] = price_list.min()
        price_stats_by_city[city]['price_by_city_max'] = price_list.max()
        price_stats_by_city[city]['price_by_city_std'] = price_list.std()
        price_stats_by_city[city]['price_by_city_count'] = len(price_list)
    for i, city in enumerate(df_train.iloc[valid_idx]['city']):
        price_stats.append((valid_idx[i], price_stats_by_city[city]))

price_stats_test = [] 
price_stats_by_city = defaultdict(dict)
for city, group in df_train.groupby('city'):
    price_list = group['keiyaku_pr']/group['tc_mseki']
    price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
    price_stats_by_city[city]['price_by_city_median'] = price_list.median()
    price_stats_by_city[city]['price_by_city_min'] = price_list.min()
    price_stats_by_city[city]['price_by_city_max'] = price_list.max()
    price_stats_by_city[city]['price_by_city_std'] = price_list.std()
    price_stats_by_city[city]['price_by_city_count'] = len(price_list)
for city in df_test['city']:
    price_stats_test.append(price_stats_by_city[city])
    
df_price_stats = pd.DataFrame([x[1] for x in sorted(price_stats, key=lambda x: x[0])])
df_price_stats_test = pd.DataFrame(price_stats_test)

df_train = pd.concat([df_train, df_price_stats], axis=1)
df_test = pd.concat([df_test, df_price_stats_test], axis=1)

## Train

In [6]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
continue_features += ['price_by_city_mean', 'price_by_city_median', 'price_by_city_min', 'price_by_city_max', 
                      'price_by_city_std', 'price_by_city_count', 'chiseki_ratio', 'lng', 'lat']
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)
        
for col in categorical_features:
    if col not in ['pj_no']:
        ct = CategoricalTransformer(min_freq=3)
        df_train[col] = ct.fit_transform(df_train[col])
        df_test[col] = ct.transform(df_test[col])
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

## LGB

In [7]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']-1e6+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']-1e6+1)
    regressor = lgb.LGBMRegressor(n_estimators=100000, learning_rate=0.01, silent=False, random_state=28,
                                  bagging_fraction=0.8, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.05, 
                                  max_depth=6, objective='fair')
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['fair'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1+1e6
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.00500358

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's fair: 0.0276382
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's fair: 0.0272857
[3]	valid_0's fair: 0.0269638
[4]	valid_0's fair: 0.0266282
[5]	valid_0's fair: 0.0263382
[6]	valid_0's fair: 0.026043
[7]	valid_0's fair: 0.0257107
[8]	valid_0's fair: 0.0253957
[9]	valid_0's fair: 0.0250906
[10]	valid_0's fair: 0.024834
[11]	valid_0's fair: 0.0245677
[12]	valid_0's fair: 0.0242851
[13]	valid_0's fair: 0.0240024
[14]	valid_0's fair: 0.0237255
[15]	valid_0's fair: 0.0234547
[16]	valid_0's fair: 0.0231914
[17]	valid_0's fair: 0.022937
[18]	valid_0's fair: 0.0226743
[19]	valid_0's fair: 0.0224165
[20]	valid_0's fair: 0.0221731
[21]	valid_0's fair: 0.0219357
[22]	valid_0's fair: 0.0217043
[23]	valid_0's fair: 0.0214763
[24]	valid_0's fair: 0.0212533
[25]	valid_0's fair: 0.0210398
[26]	valid_0's fair: 0.0208236
[27]	valid_0's fair: 0.0206229
[28]	valid_0's fair: 0.0204191
[29]	valid_0's fair: 0.0202149
[30]	valid_0's fair: 0.0200111
[31]	valid_0's fair

[284]	valid_0's fair: 0.00810324
[285]	valid_0's fair: 0.00809535
[286]	valid_0's fair: 0.00808594
[287]	valid_0's fair: 0.00807766
[288]	valid_0's fair: 0.00807008
[289]	valid_0's fair: 0.00806229
[290]	valid_0's fair: 0.00805346
[291]	valid_0's fair: 0.0080447
[292]	valid_0's fair: 0.00803479
[293]	valid_0's fair: 0.00802689
[294]	valid_0's fair: 0.00801923
[295]	valid_0's fair: 0.00801105
[296]	valid_0's fair: 0.00800438
[297]	valid_0's fair: 0.0079967
[298]	valid_0's fair: 0.00798843
[299]	valid_0's fair: 0.00798061
[300]	valid_0's fair: 0.00797248
[301]	valid_0's fair: 0.00796679
[302]	valid_0's fair: 0.00795656
[303]	valid_0's fair: 0.00794994
[304]	valid_0's fair: 0.00794398
[305]	valid_0's fair: 0.00793626
[306]	valid_0's fair: 0.00792809
[307]	valid_0's fair: 0.00792336
[308]	valid_0's fair: 0.00791489
[309]	valid_0's fair: 0.00790812
[310]	valid_0's fair: 0.00790018
[311]	valid_0's fair: 0.00789338
[312]	valid_0's fair: 0.00788341
[313]	valid_0's fair: 0.00787611
[314]	valid_

[574]	valid_0's fair: 0.00688224
[575]	valid_0's fair: 0.00687981
[576]	valid_0's fair: 0.00687722
[577]	valid_0's fair: 0.00687431
[578]	valid_0's fair: 0.00687203
[579]	valid_0's fair: 0.00686955
[580]	valid_0's fair: 0.00686812
[581]	valid_0's fair: 0.00686665
[582]	valid_0's fair: 0.00686502
[583]	valid_0's fair: 0.00686306
[584]	valid_0's fair: 0.00685961
[585]	valid_0's fair: 0.00685795
[586]	valid_0's fair: 0.00685584
[587]	valid_0's fair: 0.00685414
[588]	valid_0's fair: 0.00685137
[589]	valid_0's fair: 0.00684968
[590]	valid_0's fair: 0.0068481
[591]	valid_0's fair: 0.00684518
[592]	valid_0's fair: 0.00684261
[593]	valid_0's fair: 0.00684159
[594]	valid_0's fair: 0.00683976
[595]	valid_0's fair: 0.00683797
[596]	valid_0's fair: 0.00683594
[597]	valid_0's fair: 0.00683306
[598]	valid_0's fair: 0.00683078
[599]	valid_0's fair: 0.00682951
[600]	valid_0's fair: 0.00682764
[601]	valid_0's fair: 0.00682574
[602]	valid_0's fair: 0.0068238
[603]	valid_0's fair: 0.00682101
[604]	valid_

[825]	valid_0's fair: 0.00651012
[826]	valid_0's fair: 0.00650996
[827]	valid_0's fair: 0.00650924
[828]	valid_0's fair: 0.00650896
[829]	valid_0's fair: 0.00650759
[830]	valid_0's fair: 0.00650612
[831]	valid_0's fair: 0.00650511
[832]	valid_0's fair: 0.00650434
[833]	valid_0's fair: 0.0065033
[834]	valid_0's fair: 0.00650264
[835]	valid_0's fair: 0.00650109
[836]	valid_0's fair: 0.00650025
[837]	valid_0's fair: 0.00649915
[838]	valid_0's fair: 0.00649816
[839]	valid_0's fair: 0.00649668
[840]	valid_0's fair: 0.00649559
[841]	valid_0's fair: 0.00649444
[842]	valid_0's fair: 0.00649355
[843]	valid_0's fair: 0.00649297
[844]	valid_0's fair: 0.00649158
[845]	valid_0's fair: 0.00649022
[846]	valid_0's fair: 0.00648994
[847]	valid_0's fair: 0.00648947
[848]	valid_0's fair: 0.00648888
[849]	valid_0's fair: 0.00648809
[850]	valid_0's fair: 0.00648684
[851]	valid_0's fair: 0.00648638
[852]	valid_0's fair: 0.00648492
[853]	valid_0's fair: 0.00648402
[854]	valid_0's fair: 0.00648306
[855]	valid

...

[2276]	valid_0's fair: 0.00524611
[2277]	valid_0's fair: 0.00524593
[2278]	valid_0's fair: 0.0052459
[2279]	valid_0's fair: 0.00524671
[2280]	valid_0's fair: 0.00524647
[2281]	valid_0's fair: 0.00524606
[2282]	valid_0's fair: 0.00524602
[2283]	valid_0's fair: 0.00524579
[2284]	valid_0's fair: 0.0052455
[2285]	valid_0's fair: 0.00524549
[2286]	valid_0's fair: 0.00524566
[2287]	valid_0's fair: 0.00524546
[2288]	valid_0's fair: 0.00524549
[2289]	valid_0's fair: 0.00524566
[2290]	valid_0's fair: 0.00524567
[2291]	valid_0's fair: 0.00524592
[2292]	valid_0's fair: 0.00524654
[2293]	valid_0's fair: 0.00524646
[2294]	valid_0's fair: 0.00524652
[2295]	valid_0's fair: 0.0052463
[2296]	valid_0's fair: 0.00524611
[2297]	valid_0's fair: 0.00524645
[2298]	valid_0's fair: 0.0052462
[2299]	valid_0's fair: 0.00524639
[2300]	valid_0's fair: 0.00524623
[2301]	valid_0's fair: 0.00524652
[2302]	valid_0's fair: 0.00524657
[2303]	valid_0's fair: 0.0052462
[2304]	valid_0's fair: 0.00524611
[2305]	valid_0's fa

[2524]	valid_0's fair: 0.00523386
[2525]	valid_0's fair: 0.00523383
[2526]	valid_0's fair: 0.005234
[2527]	valid_0's fair: 0.00523417
[2528]	valid_0's fair: 0.00523424
[2529]	valid_0's fair: 0.00523428
[2530]	valid_0's fair: 0.00523423
[2531]	valid_0's fair: 0.0052342
[2532]	valid_0's fair: 0.00523443
[2533]	valid_0's fair: 0.00523396
[2534]	valid_0's fair: 0.00523351
[2535]	valid_0's fair: 0.0052333
[2536]	valid_0's fair: 0.0052332
[2537]	valid_0's fair: 0.00523285
[2538]	valid_0's fair: 0.00523291
[2539]	valid_0's fair: 0.00523328
[2540]	valid_0's fair: 0.00523255
[2541]	valid_0's fair: 0.00523278
[2542]	valid_0's fair: 0.00523274
[2543]	valid_0's fair: 0.00523278
[2544]	valid_0's fair: 0.00523302
[2545]	valid_0's fair: 0.00523291
[2546]	valid_0's fair: 0.00523266
[2547]	valid_0's fair: 0.00523282
[2548]	valid_0's fair: 0.00523266
[2549]	valid_0's fair: 0.0052325
[2550]	valid_0's fair: 0.00523235
[2551]	valid_0's fair: 0.0052321
[2552]	valid_0's fair: 0.00523174
[2553]	valid_0's fair

[2769]	valid_0's fair: 0.00522731
[2770]	valid_0's fair: 0.00522699
[2771]	valid_0's fair: 0.00522647
[2772]	valid_0's fair: 0.00522649
[2773]	valid_0's fair: 0.00522635
[2774]	valid_0's fair: 0.00522632
[2775]	valid_0's fair: 0.00522655
[2776]	valid_0's fair: 0.00522616
[2777]	valid_0's fair: 0.00522611
[2778]	valid_0's fair: 0.00522648
[2779]	valid_0's fair: 0.00522663
[2780]	valid_0's fair: 0.00522675
[2781]	valid_0's fair: 0.00522627
[2782]	valid_0's fair: 0.00522586
[2783]	valid_0's fair: 0.00522548
[2784]	valid_0's fair: 0.00522552
[2785]	valid_0's fair: 0.00522552
[2786]	valid_0's fair: 0.00522512
[2787]	valid_0's fair: 0.00522533
[2788]	valid_0's fair: 0.00522538
[2789]	valid_0's fair: 0.00522505
[2790]	valid_0's fair: 0.00522524
[2791]	valid_0's fair: 0.00522526
[2792]	valid_0's fair: 0.00522514
[2793]	valid_0's fair: 0.00522509
[2794]	valid_0's fair: 0.00522502
[2795]	valid_0's fair: 0.00522503
[2796]	valid_0's fair: 0.00522532
[2797]	valid_0's fair: 0.00522531
[2798]	valid_0

[3020]	valid_0's fair: 0.00522817
[3021]	valid_0's fair: 0.00522774
[3022]	valid_0's fair: 0.00522751
[3023]	valid_0's fair: 0.00522777
[3024]	valid_0's fair: 0.00522798
[3025]	valid_0's fair: 0.00522696
[3026]	valid_0's fair: 0.00522725
[3027]	valid_0's fair: 0.0052275
[3028]	valid_0's fair: 0.00522723
[3029]	valid_0's fair: 0.00522725
[3030]	valid_0's fair: 0.00522738
[3031]	valid_0's fair: 0.00522742
[3032]	valid_0's fair: 0.00522747
[3033]	valid_0's fair: 0.00522732
[3034]	valid_0's fair: 0.00522717
[3035]	valid_0's fair: 0.0052273
[3036]	valid_0's fair: 0.00522704
[3037]	valid_0's fair: 0.00522716
[3038]	valid_0's fair: 0.00522719
[3039]	valid_0's fair: 0.00522719
[3040]	valid_0's fair: 0.00522699
[3041]	valid_0's fair: 0.00522668
[3042]	valid_0's fair: 0.00522681
[3043]	valid_0's fair: 0.00522661
[3044]	valid_0's fair: 0.00522632
[3045]	valid_0's fair: 0.0052263
[3046]	valid_0's fair: 0.00522636
[3047]	valid_0's fair: 0.00522635
[3048]	valid_0's fair: 0.00522642
[3049]	valid_0's 

[3271]	valid_0's fair: 0.00522707
[3272]	valid_0's fair: 0.00522679
[3273]	valid_0's fair: 0.00522697
[3274]	valid_0's fair: 0.00522654
[3275]	valid_0's fair: 0.00522681
[3276]	valid_0's fair: 0.00522636
[3277]	valid_0's fair: 0.00522681
[3278]	valid_0's fair: 0.00522744
[3279]	valid_0's fair: 0.0052277
[3280]	valid_0's fair: 0.00522755
[3281]	valid_0's fair: 0.00522778
[3282]	valid_0's fair: 0.00522732
[3283]	valid_0's fair: 0.00522724
[3284]	valid_0's fair: 0.00522716
[3285]	valid_0's fair: 0.00522768
[3286]	valid_0's fair: 0.00522731
[3287]	valid_0's fair: 0.00522731
[3288]	valid_0's fair: 0.0052274
[3289]	valid_0's fair: 0.00522776
[3290]	valid_0's fair: 0.00522737
[3291]	valid_0's fair: 0.00522749
[3292]	valid_0's fair: 0.00522729
[3293]	valid_0's fair: 0.00522741
[3294]	valid_0's fair: 0.00522745
Early stopping, best iteration is:
[2794]	valid_0's fair: 0.00522502
5-fold cv mean l2 0.00500358


In [8]:
tmp = list(df_train)
tmp.remove('keiyaku_pr')

In [9]:
list(zip(tmp, regressor.feature_importances_))

[('pj_no', 1),
 ('tc_mseki', 3521),
 ('tt_mseki', 4807),
 ('levelplan', 373),
 ('fukuin', 1401),
 ('road_st', 1),
 ('magutchi', 5309),
 ('setsudo_hi', 2090),
 ('setsudo_kj', 543),
 ('jigata', 266),
 ('hiatari', 898),
 ('niwasaki', 3239),
 ('garage', 503),
 ('kobetsu1', 939),
 ('kobetsu2', 16),
 ('kobetsu3', 0),
 ('kobetsu4', 0),
 ('jukyo', 605),
 ('chiseki_js_hb', 1264),
 ('chiseki_kb_hb', 1054),
 ('kempei1', 37),
 ('kempei2', 31),
 ('yoseki1', 118),
 ('yoseki2', 138),
 ('josui', 0),
 ('gesui', 79),
 ('gas', 384),
 ('usui', 40),
 ('tateuri_su', 381),
 ('tochiuri_su', 147),
 ('joken_su', 3),
 ('hy1f_date_su', 0),
 ('hy2f_date_su', 209),
 ('hy3f_date_su', 29),
 ('road1_sb', 55),
 ('road1_fi', 981),
 ('road1_mg', 1655),
 ('road2_sb', 100),
 ('road2_fi', 619),
 ('road2_mg', 1444),
 ('road3_sb', 18),
 ('road3_fi', 107),
 ('road3_mg', 190),
 ('road4_sb', 2),
 ('road4_fi', 27),
 ('road4_mg', 21),
 ('kaoku_hb', 1227),
 ('yheki_kotei', 752),
 ('yheki_umu', 31),
 ('yheki_yohi', 30),
 ('gk_sho_ky